In [ ]:
＃安装paddleslim
!pip install paddleslim


In [ ]:
＃导入依赖
import paddle
import paddle.fluid as fluid
import paddleslim as slim
import numpy as np

In [ ]:
＃构建模型
use_gpu = fluid.is_compiled_with_cuda()
exe, train_program, val_program, inputs, outputs = slim.models.image_classification("MobileNet", [1, 28, 28], 10, use_gpu=use_gpu)
place = fluid.CUDAPlace(0) if fluid.is_compiled_with_cuda() else fluid.CPUPlace()

In [ ]:
＃定义输入数据
import paddle.dataset.mnist as reader
train_reader = paddle.batch(
        reader.train(), batch_size=128, drop_last=True)
test_reader = paddle.batch(
        reader.test(), batch_size=128, drop_last=True)
data_feeder = fluid.DataFeeder(inputs, place)

In [ ]:
＃训练和测试　　先定义训练和测试函数，正常训练和量化训练时只需要调用函数即可。在训练函数中执行了一个epoch的训练，因为MNIST数据集数据较少，一个epoch就可将top1精度训练到95%以上。
def train(prog):
    iter = 0
    for data in train_reader():
        acc1, acc5, loss = exe.run(prog, feed=data_feeder.feed(data), fetch_list=outputs)
        if iter % 100 == 0:
            print('train iter={}, top1={}, top5={}, loss={}'.format(iter, acc1.mean(), acc5.mean(), loss.mean()))
        iter += 1
        
def test(prog):
    iter = 0
    res = [[], []]
    for data in test_reader():
        acc1, acc5, loss = exe.run(prog, feed=data_feeder.feed(data), fetch_list=outputs)
        if iter % 100 == 0:
            print('test iter={}, top1={}, top5={}, loss={}'.format(iter, acc1.mean(), acc5.mean(), loss.mean()))
        res[0].append(acc1.mean())
        res[1].append(acc5.mean())
        iter += 1
    print('final test result top1={}, top5={}'.format(np.array(res[0]).mean(), np.array(res[1]).mean()))

In [ ]:
train(train_program)
train iter=0, top1=0.109375, top5=0.4921875, loss=2.6497092247
train iter=100, top1=0.9296875, top5=1.0, loss=0.231126576662
train iter=200, top1=0.9609375, top5=1.0, loss=0.163963362575
train iter=300, top1=0.96875, top5=0.9921875, loss=0.169646322727
# train iter=400, top1=0.9453125, top5=1.0, loss=0.185211241245

＃调用test函# 测试分类网络，val_program是在第2步：构建网络中定义的。

In [ ]:
test(val_program)
test iter=0, top1=0.984375, top5=1.0, loss=0.0616650208831
final test result top1=0.971654653549, top5=0.999098539352

In [ ]:
＃量化模型
place = exe.place
quant_program = slim.quant.quant_aware(train_program, exe.place, for_test=False)
val_quant_program = slim.quant.quant_aware(val_program, exe.place, for_test=True)

In [ ]:
＃训练和测试量化后的模型
train(quant_program)
train iter=0, top1=0.96875, top5=1.0, loss=0.0923340842128
train iter=100, top1=0.9765625, top5=1.0, loss=0.0619144812226
train iter=200, top1=0.984375, top5=1.0, loss=0.0748545378447
train iter=300, top1=0.984375, top5=1.0, loss=0.0823880583048
# train iter=400, top1=0.9453125, top5=1.0, loss=0.136059805751

测试量化后的模型# ，和3.2 训练和测试中得到的测试结果相比，精度相近，达到了无损量化。

In [ ]:
test(val_quant_program)
test iter=0, top1=0.984375, top5=1.0, loss=0.0461478419602
# final test result top1=0.974258840084, top5=0.999098539352